# Calibración de offset

In [ ]:
import threading
import time
import csv
import numpy as np
import sounddevice as sd
import serial

In [ ]:
SLM_COM_PORT = "COMX"
SLM_BAUDRATE = 115200
SLM_PERIOD = 0.125

# System volume level required to reach the offset reference value (i.e. 94 dB) 
OFFSET_REF_VOLUME = 0.95
# Reference dB value
REF_DB = 94.0

In [ ]:
# Function to receive serial data and store it
def receive_serial_data(data):
    ser = serial.Serial(SLM_COM_PORT, SLM_BAUDRATE)  # Replace 'COM1' with your serial port
    while not terminate_event.is_set():
        received_data = float(ser.readline().decode().strip())
        data.append(received_data)
    ser.close()

# Function to play audio sine waves
def play_audio_sine_wave():
    # Get the default audio playback device
    devices = AudioUtilities.GetSpeakers()
    interface = devices.Activate(
        IAudioEndpointVolume._iid_, CLSCTX_ALL, None)

    # Create a volume control interface and set the volume
    volume_controller = cast(interface, POINTER(IAudioEndpointVolume))
    volume_controller.SetMasterVolumeLevelScalar(OFFSET_REF_VOLUME, None)

    duration = 15  #s
    t = np.arange(0, duration, 1 / 88200)
    sine_wave = np.sin(2 * np.pi * 1000 * t)  # 1 kHz sine wave

    # Wait before playing the sine wave
    time.sleep(3) #s

    sd.play(sine_wave, sample_rate, blocking=True)

    # Wait after playback is done
    time.sleep(3) #s
    # Send a signal to the other thread to terminate
    terminate_event.set()

In [ ]:
# Create a list to store received serial data
serial_data = []

# Event to signal termination
terminate_event = threading.Event()

# Create the serial data receiving thread
serial_thread = threading.Thread(target=receive_serial_data, args=(serial_data,))
serial_thread.daemon = True

# Create the audio playback thread
audio_thread = threading.Thread(target=play_audio_sine_wave)
audio_thread.daemon = True

## Ejecutar prueba

In [ ]:
# Start both threads
serial_thread.start()
audio_thread.start()

# Wait for the threads to finish
serial_thread.join()
audio_thread.join()

## Analizar datos

In [ ]:
# Inspect the dB samples
print(serial_data)
# Take the average measurement
avg = sum(serial_data)/len(serial_data)

# Calculate the offset
offset = REF_DB - avg
print(offset)